In [1]:
import pandas as pd
import numpy as np

In [2]:
data1=pd.read_csv("D:\Final Year Project\FML\\1st Model\\1.csv")

In [3]:
data1

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,54.00,6.00,64.00,0.329807,0.329807,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334383e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
1,0.000000,57.04,6.33,64.00,4.290556,4.290556,0.0,0.0,0.0,0.0,...,2.822973,57.04,8.292607e+07,9.5,10.464666,4.010353,160.987842,0.05,141.55,DoS-TCP_Flood
2,0.000000,0.00,1.00,64.00,33.396799,33.396799,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.312799e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,DDoS-ICMP_Flood
3,0.328175,76175.00,17.00,64.00,4642.133010,4642.133010,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.301570e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,DoS-UDP_Flood
4,0.117320,101.73,6.11,65.91,6.202211,6.202211,0.0,0.0,1.0,0.0,...,23.113111,57.88,8.297300e+07,9.5,11.346876,32.716243,3016.808286,0.19,141.55,DoS-SYN_Flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238682,0.000000,54.00,6.00,64.00,3.049186,3.049186,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334449e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-RSTFINFlood
238683,0.000000,54.00,6.00,64.00,183.433732,183.433732,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331392e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-PSHACK_Flood
238684,0.000785,56.29,6.11,64.00,306.952216,306.952216,0.0,0.0,1.0,0.0,...,0.140764,54.21,8.308883e+07,9.5,10.395538,0.200659,0.671167,0.03,141.55,DDoS-SYN_Flood
238685,0.000901,72.09,6.11,64.64,158.475986,158.475986,0.0,0.0,0.0,0.0,...,2.450404,55.48,8.333177e+07,9.5,10.456522,3.475801,55.994224,0.17,141.55,DDoS-PSHACK_Flood


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from skfeature.function.information_theoretical_based import MRMR

In [5]:
X,y=data1.iloc[:,0:46],data1.iloc[:,46]

In [6]:
len(X)

238687

In [7]:
print(y.value_counts().sum())

238687


In [10]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 238687 entries, 0 to 238686
Series name: label
Non-Null Count   Dtype 
--------------   ----- 
238687 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


In [11]:
mapping = {'BenignTraffic': 0}
# Set all other values to 1
mapping.update({key: 1 for key in y.unique() if key != 'BenignTraffic'})

# Map the values using the dictionary
y = y.map(mapping)

In [13]:
y.sum()

233087

### Feature Selection with RFE


In [14]:
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier()

# Perform Recursive Feature Elimination (RFE)
rfe = RFE(rf_classifier)  # 'k' is the number of features you want to select
rfe.fit(X, y)

# Get the selected features using RFE
selected_features_rfe=X.iloc[:,rfe.support_]

In [17]:
X.iloc[:,rfe.support_].columns

Index(['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate',
       'Srate', 'syn_count', 'urg_count', 'rst_count', 'HTTPS', 'Tot sum',
       'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight'],
      dtype='object')

#### Feature Selection with MRMR:

In [18]:
# Perform Minimum Redundancy Maximum Relevance (mRMR)
selected_features_mrmr = MRMR.mrmr(X.values, y.values)

In [20]:
selected_features_mrmr[0:20]

array([45, 33, 32,  5, 23, 25,  0, 22, 20, 34, 17, 16,  9,  8, 12, 15, 42,
       18,  4, 10])

In [23]:
mrmr_data=X.iloc[:,selected_features_mrmr[0:20]]

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238687 entries, 0 to 238686
Data columns (total 46 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   flow_duration    238687 non-null  float64
 1   Header_Length    238687 non-null  float64
 2   Protocol Type    238687 non-null  float64
 3   Duration         238687 non-null  float64
 4   Rate             238687 non-null  float64
 5   Srate            238687 non-null  float64
 6   Drate            238687 non-null  float64
 7   fin_flag_number  238687 non-null  float64
 8   syn_flag_number  238687 non-null  float64
 9   rst_flag_number  238687 non-null  float64
 10  psh_flag_number  238687 non-null  float64
 11  ack_flag_number  238687 non-null  float64
 12  ece_flag_number  238687 non-null  float64
 13  cwr_flag_number  238687 non-null  float64
 14  ack_count        238687 non-null  float64
 15  syn_count        238687 non-null  float64
 16  fin_count        238687 non-null  floa

In [24]:
mrmr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238687 entries, 0 to 238686
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Weight           238687 non-null  float64
 1   Tot sum          238687 non-null  float64
 2   LLC              238687 non-null  float64
 3   Srate            238687 non-null  float64
 4   SMTP             238687 non-null  float64
 5   IRC              238687 non-null  float64
 6   flow_duration    238687 non-null  float64
 7   Telnet           238687 non-null  float64
 8   HTTPS            238687 non-null  float64
 9   Min              238687 non-null  float64
 10  urg_count        238687 non-null  float64
 11  fin_count        238687 non-null  float64
 12  rst_flag_number  238687 non-null  float64
 13  syn_flag_number  238687 non-null  float64
 14  ece_flag_number  238687 non-null  float64
 15  syn_count        238687 non-null  float64
 16  Radius           238687 non-null  floa

In [25]:
selected_features_rfe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238687 entries, 0 to 238686
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   flow_duration  238687 non-null  float64
 1   Header_Length  238687 non-null  float64
 2   Protocol Type  238687 non-null  float64
 3   Duration       238687 non-null  float64
 4   Rate           238687 non-null  float64
 5   Srate          238687 non-null  float64
 6   syn_count      238687 non-null  float64
 7   urg_count      238687 non-null  float64
 8   rst_count      238687 non-null  float64
 9   HTTPS          238687 non-null  float64
 10  Tot sum        238687 non-null  float64
 11  Min            238687 non-null  float64
 12  Max            238687 non-null  float64
 13  AVG            238687 non-null  float64
 14  Std            238687 non-null  float64
 15  Tot size       238687 non-null  float64
 16  IAT            238687 non-null  float64
 17  Number         238687 non-nul

In [26]:
# Combine selected features from RFE and MRMR
RF_MRMR_data=pd.concat([selected_features_rfe,mrmr_data],axis=1)


In [27]:
RF_MRMR_data = RF_MRMR_data.loc[:,~RF_MRMR_data.columns.duplicated()]

In [30]:
RF_MRMR_data .columns

Index(['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate',
       'Srate', 'syn_count', 'urg_count', 'rst_count', 'HTTPS', 'Tot sum',
       'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight', 'LLC', 'SMTP', 'IRC',
       'Telnet', 'fin_count', 'rst_flag_number', 'syn_flag_number',
       'ece_flag_number', 'psh_flag_number'],
      dtype='object')

### Class Balancing

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assuming 'X' is your feature matrix and 'y' is your target variable
X_normalized = StandardScaler().fit_transform(X)

# Split the dataset into train and test sets with an 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

### Techniques

 Random Oversampling (ROS):

In [32]:
from imblearn.over_sampling import RandomOverSampler

# Apply Random Oversampling to the training set
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [33]:
y_resampled.value_counts()

1    233087
0    233087
Name: label, dtype: int64

Balanced Random Forest Classifier sampling (BRFC):

In [34]:
from imblearn.ensemble import BalancedRandomForestClassifier

# Create a BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier on the original imbalanced data
brfc.fit(X_train, y_train)

# Make predictions on the test data
y_pred = brfc.predict(X_test)

c:\Python311\Lib\site-packages\imblearn\ensemble\_forest.py:546: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
c:\Python311\Lib\site-packages\imblearn\ensemble\_forest.py:558: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


Base+BRFC

In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score

# Calculate Precision
precision = precision_score(y_test, y_pred, average='weighted')

# Calculate Recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test, y_pred)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.9923060267523797
Recall: 0.9885416230256818
F1 Score: 0.9896338887106656
Accuracy: 0.9885416230256818
Cohen's Kappa Score: 0.7977828625695117


In [36]:
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [37]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [38]:
y_pred_resampled=rf_classifier.predict(X_test_resampled)

Base+ROS

In [39]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score
# Calculate Precision
precision = precision_score(y_test_resampled, y_pred_resampled, average='weighted')

# Calculate Recall
recall = recall_score(y_test_resampled, y_pred_resampled, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test_resampled, y_pred_resampled, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test_resampled, y_pred_resampled)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test_resampled, y_pred_resampled)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.9987589170589931
Recall: 0.998755832037325
F1 Score: 0.9987558285009288
Accuracy: 0.998755832037325
Cohen's Kappa Score: 0.9975116549394033


RF_MRMR

In [40]:
X_train_rfmrmr, X_test_rfmrmr, y_train_rfmrmr, y_test_rfmrmr = train_test_split(RF_MRMR_data, y, test_size=0.2, random_state=42)

In [41]:
rf_classifier.fit(X_train_rfmrmr, y_train_rfmrmr)

RandomForestClassifier(random_state=42)

In [42]:
y_pred_rfmrmr=rf_classifier.predict(X_test_rfmrmr)

In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score

# Calculate Precision
precision = precision_score(y_test_rfmrmr, y_pred_rfmrmr, average='weighted')

# Calculate Recall
recall = recall_score(y_test_rfmrmr, y_pred_rfmrmr, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test_rfmrmr, y_pred_rfmrmr, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test_rfmrmr, y_pred_rfmrmr)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test_rfmrmr, y_pred_rfmrmr)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.9967070424281634
Recall: 0.9966693200385437
F1 Score: 0.9966858647917821
Accuracy: 0.9966693200385437
Cohen's Kappa Score: 0.9279093574813504


RF_MRMR + ROS

In [44]:
X_resampled_rfmrmr, y_resampled_rfmrmr = ros.fit_resample(RF_MRMR_data, y)

In [45]:
X_train_rfmrmr_resampled, X_test_rfmrmr_resampled, y_train_rfmrmr_resampled, y_test_rfmrmr_resampled = train_test_split(X_resampled_rfmrmr, y_resampled_rfmrmr, test_size=0.2, random_state=42)

In [46]:
rf_classifier.fit(X_train_rfmrmr_resampled, y_train_rfmrmr_resampled)

RandomForestClassifier(random_state=42)

In [47]:
y_pred_rfmrmr_resampled=rf_classifier.predict(X_test_rfmrmr_resampled)

In [48]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score

# Calculate Precision
precision = precision_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled, average='weighted')

# Calculate Recall
recall = recall_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.9986842210031718
Recall: 0.9986807529361291
F1 Score: 0.998680748829378
Accuracy: 0.9986807529361291
Cohen's Kappa Score: 0.997361495773553


In [49]:
from joblib import dump, load
dump(rf_classifier, '1st_Model.joblib')

['1st_Model.joblib']